In [3]:
import io
import shutil
import os
from functools import reduce
from tempfile import TemporaryDirectory
import logging
from urllib.request import urlopen, Request
from zipfile import ZipFile
import pandas as pd
import sandy

In [4]:
from sandy.libraries import (
N_FILES_ENDFB_71_IAEA,
N_FILES_JEFF_32_NEA,
N_FILES_JEFF_33_IAEA,
N_FILES_JEFF_40T0_NEA,
N_FILES_ENDFB_80_IAEA,
N_FILES_JENDL_40U_IAEA,
URL_N_ENDFB_71_IAEA,
URL_N_JEFF_32_NEA,
URL_N_JEFF_33_IAEA,
URL_N_JEFF_40T0_NEA,
URL_N_ENDFB_80_IAEA,
URL_N_JENDL_40U_IAEA,
URL_NFPY_ENDFB_80_IAEA,
NFPY_FILES_ENDFB_80_IAEA,
URL_NFPY_ENDFB_71_IAEA,
NFPY_FILES_ENDFB_71_IAEA,
URL_NFPY_JENDL_40U_IAEA,
NFPY_FILES_JENDL_40U_IAEA,
URL_NFPY_JEFF_33_IAEA,
NFPY_FILES_JEFF_33_IAEA,
URL_DECAY_ENDFB_71_IAEA,
DECAY_FILES_ENDFB_71_IAEA,
URL_DECAY_ENDFB_80_IAEA,
DECAY_FILES_ENDFB_80_IAEA,
URL_DECAY_JEFF_33_IAEA,
DECAY_FILES_JEFF_33_IAEA,
)

__author__ = "Luca Fiorito"
__all__ = [
"Endf6",
"get_endf6_file",
]
pd.options.display.float_format = '{:.5e}'.format

In [7]:
# Input for test
library = 'jeff_33'
zam = 922350
input_endf6 = sandy.get_endf6_file(library, 'decay', zam)
percentage = 5/100

In [8]:
# Decay Data instance
decay_data_instance = sandy.DecayData.from_endf6(input_endf6)

In [8]:
# Method proposal - get_decayconstant - restituisce un pd.Series con le decay constants
# Input: (self, with_uncertainty=True)
decay_constants = {zam: dic['decay_constant'] for zam, dic in decay_data_instance.data.items()} # where decay_data_instance will be self in the method
series = pd.Series(decay_constants, name = 'lambda')
series.index.name = 'ZAM'
with_uncertainty = False
if not with_uncertainty: # to take as input, default = True
    print(series)
    #return series
else:
    uncertainty = get_decayconstant_uncertainty
    a = pd.concat([series, uncertainty], axis=1, keys=['lambda', 'uncertainty'])
    #return a

ZAM
922350   3.12085e-17
Name: lambda, dtype: float64


In [5]:
# print(f'tipo {type(decay_data_instance.data)}\n\n variable {decay_data_instance.data}')

decay_data_instance.data.items()

dict_items([(922350, {'half_life': 2.22102e+16, 'decay_constant': 3.120850692744529e-17, 'stable': False, 'spin': 3.5, 'parity': -1.0, 'decay_energy': {'beta': 50671.7, 'gamma': 163616.0, 'alpha': 4464600.0, 'total': 4678887.7}, 'decay_energy_uncertainties': {'beta': 4291.63, 'gamma': 1708.01, 'alpha': 163255.0, 'total': 163320.33067324167}, 'decay_modes': {'40x0': {'decay_products': {902310: 1.0, 20040: 1.0}, 'branching_ratio': 1.0, 'branching_ratio_uncertainty': 0.0001}, '60x0': {'decay_products': {}, 'branching_ratio': 7.2e-11, 'branching_ratio_uncertainty': 2.1e-11}}})])

In [ ]:
# Method proposal - _get_decayconstant_uncertainty - restituisce un pd.Series con le incertezze
# Assuming from_endf6 method to be updated including decay_constant_uncertainty- ###chiedere### 
# Input: (self)

uncertainty = {zam: dic['decay_constant_uncertainty'] for zam, dic in decay_data_instance.data.item()}
series = pd.Series(uncertainty, name = 'lambda uncertainty')
series.index.name = 'ZAM'

#return serie

In [ ]:
# Method proposal - custom_perturbation_decayconstant - restituisce un pd.Series con le decayconstants perturbate e relative incertezze
# Input: (self, zam, percentage)
series = get_decay_constant()
series['lambda'][zam] = series['lambda'][zam]*(1+percentage)
#return series

# Esempio

In [14]:
series[zam] = series[zam]*(1+percentage)

print(series)

ZAM
922350   3.27689e-17
Name: lambda, dtype: float64


In [11]:
halflive=sandy.DecayData.get_halflives(decay_data_instance)

print(halflive)

ZAM
922350   2.22102e+16
Name: T1/2, dtype: float64


In [ ]:
#Method proposal -to endf6
